In [1]:
import json
import random
import re
from pathlib import Path
from itertools import permutations
from tqdm import tqdm

In [2]:
# Định nghĩa đường dẫn
input_file = Path("../data/sft_dataset_vnhsge/train_sft.jsonl")
output_file = Path("../data/sft_dataset_vnhsge/train_sft_augmented.jsonl")

print(f"Input file: {input_file}")
print(f"Output file: {output_file}")
print(f"File exists: {input_file.exists()}")

Input file: ..\data\sft_dataset_vnhsge\train_sft.jsonl
Output file: ..\data\sft_dataset_vnhsge\train_sft_augmented.jsonl
File exists: True


In [3]:
def parse_question(content):
    """
    Parse câu hỏi trắc nghiệm để tách question và các options A, B, C, D
    """
    lines = content.strip().split('\n')
    
    # Tìm câu hỏi (dòng đầu tiên không có A., B., C., D.)
    question_parts = []
    options = {}
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # Kiểm tra xem có phải là option không
        match = re.match(r'^([A-D])\.(.*)', line)
        if match:
            option_letter = match.group(1)
            option_text = match.group(2).strip()
            options[option_letter] = option_text
        else:
            question_parts.append(line)
    
    question = ' '.join(question_parts)
    return question, options

# Test function
test_content = """Câu 81: Có thể sử dụng hóa chất nào sau đây để phát hiện quá trình hô hấp ở thực vật thải ra khí CO2? 
A. Dung dịch NaCl. 
B. Dung dịch Ca(OH)2. 
C. Dung dịch KCl. 
D. Dung dịch H2SO4."""

q, opts = parse_question(test_content)
print("Question:", q)
print("Options:", opts)

Question: Câu 81: Có thể sử dụng hóa chất nào sau đây để phát hiện quá trình hô hấp ở thực vật thải ra khí CO2?
Options: {'A': 'Dung dịch NaCl.', 'B': 'Dung dịch Ca(OH)2.', 'C': 'Dung dịch KCl.', 'D': 'Dung dịch H2SO4.'}


In [4]:
def shuffle_options(question, options, correct_answer):
    """
    Hoán đổi vị trí các options và cập nhật đáp án đúng
    
    Args:
        question: Câu hỏi gốc
        options: Dict {"A": "...", "B": "...", "C": "...", "D": "..."}
        correct_answer: Đáp án đúng ban đầu ("A", "B", "C", hoặc "D")
    
    Returns:
        new_content: Nội dung câu hỏi mới với options đã hoán đổi
        new_answer: Đáp án đúng mới sau khi hoán đổi
    """
    if len(options) != 4:
        # Nếu không đủ 4 options, trả về nguyên bản
        return None, None
    
    # Tạo danh sách các option values
    option_values = [options["A"], options["B"], options["C"], options["D"]]
    
    # Shuffle các options
    shuffled_indices = list(range(4))
    random.shuffle(shuffled_indices)
    
    # Tạo options mới
    new_options = {}
    letters = ["A", "B", "C", "D"]
    
    for new_idx, old_idx in enumerate(shuffled_indices):
        new_options[letters[new_idx]] = option_values[old_idx]
        
        # Tìm vị trí mới của đáp án đúng
        if letters[old_idx] == correct_answer:
            new_answer = letters[new_idx]
    
    # Tạo nội dung mới
    new_content = question + " \n"
    for letter in letters:
        new_content += f"{letter}. {new_options[letter]} \n"
    
    return new_content.strip(), new_answer

# Test function
test_options = {
    "A": "Dung dịch NaCl.",
    "B": "Dung dịch Ca(OH)2.",
    "C": "Dung dịch KCl.",
    "D": "Dung dịch H2SO4."
}
test_question = "Câu 81: Có thể sử dụng hóa chất nào sau đây để phát hiện quá trình hô hấp ở thực vật thải ra khí CO2?"

new_content, new_answer = shuffle_options(test_question, test_options, "B")
print("Original answer: B")
print("New answer:", new_answer)
print("\nNew content:")
print(new_content)

Original answer: B
New answer: C

New content:
Câu 81: Có thể sử dụng hóa chất nào sau đây để phát hiện quá trình hô hấp ở thực vật thải ra khí CO2? 
A. Dung dịch KCl. 
B. Dung dịch NaCl. 
C. Dung dịch Ca(OH)2. 
D. Dung dịch H2SO4.


In [5]:
def augment_data(input_file, output_file, num_augmentations=2):
    """
    Augment dữ liệu bằng cách hoán đổi vị trí các options
    
    Args:
        input_file: File JSONL đầu vào
        output_file: File JSONL đầu ra
        num_augmentations: Số lượng phiên bản augmented cho mỗi câu hỏi
    """
    # Đọc dữ liệu gốc
    original_data = []
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            original_data.append(json.loads(line))
    
    print(f"Loaded {len(original_data)} original samples")
    
    # Tạo dữ liệu augmented
    augmented_data = []
    skipped = 0
    
    for item in tqdm(original_data, desc="Augmenting data"):
        # Giữ lại sample gốc
        augmented_data.append(item)
        
        # Parse câu hỏi
        user_content = item["messages"][1]["content"]
        correct_answer = json.loads(item["messages"][2]["content"])["answer"]
        
        try:
            question, options = parse_question(user_content)
            
            if len(options) != 4:
                skipped += 1
                continue
            
            # Tạo các phiên bản augmented
            for _ in range(num_augmentations):
                new_content, new_answer = shuffle_options(question, options, correct_answer)
                
                if new_content is None:
                    continue
                
                # Tạo sample mới
                new_item = {
                    "messages": [
                        item["messages"][0],  # System message giữ nguyên
                        {"role": "user", "content": new_content},
                        {"role": "assistant", "content": json.dumps({"answer": new_answer}, ensure_ascii=False)}
                    ],
                    "id": f"{item['id']}_aug_{len(augmented_data)}",
                    "subject": item["subject"]
                }
                augmented_data.append(new_item)
        
        except Exception as e:
            print(f"\nError processing item {item['id']}: {e}")
            skipped += 1
            continue
    
    print(f"\nTotal samples after augmentation: {len(augmented_data)}")
    print(f"Skipped: {skipped}")
    print(f"Augmentation ratio: {len(augmented_data) / len(original_data):.2f}x")
    
    # Shuffle dữ liệu
    random.shuffle(augmented_data)
    
    # Ghi ra file
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in augmented_data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"\nSaved augmented data to {output_file}")
    return augmented_data

In [6]:
# Thực hiện augmentation
# num_augmentations=2 có nghĩa là mỗi câu hỏi gốc sẽ tạo ra 2 phiên bản augmented
# Tổng cộng mỗi câu hỏi sẽ có 3 phiên bản (1 gốc + 2 augmented)

augmented_data = augment_data(
    input_file=input_file,
    output_file=output_file,
    num_augmentations=2  # Có thể điều chỉnh số này
)

Loaded 1573 original samples


Augmenting data:   0%|          | 0/1573 [00:00<?, ?it/s]

Augmenting data: 100%|██████████| 1573/1573 [00:00<00:00, 30980.95it/s]


Total samples after augmentation: 4719
Skipped: 0
Augmentation ratio: 3.00x

Saved augmented data to ..\data\sft_dataset_vnhsge\train_sft_augmented.jsonl


In [7]:
# Kiểm tra một vài samples
print("\n" + "="*80)
print("Sample augmented data:")
print("="*80)

for i in range(3):
    sample = augmented_data[i]
    print(f"\n--- Sample {i+1} (ID: {sample['id']}) ---")
    print(f"Subject: {sample['subject']}")
    print(f"\nUser content:\n{sample['messages'][1]['content']}")
    print(f"\nAssistant: {sample['messages'][2]['content']}")


Sample augmented data:

--- Sample 1 (ID: MET_Phy_IE_2023_5) ---
Subject: Physics

User content:
Câu 5. Hai dao động điều hòa cùng tần số có pha ban đầu là $\varphi_{1}$ và $\varphi_{2}$. Hai dao động này cùng pha khi
A. $\varphi_{2}-\varphi_{1}=(2 n+1) \pi$ với $n=0, \pm 1, \pm 2, \ldots$.
B. $\varphi_{2}-\varphi_{1}=2 n \pi$ với $n=0, \pm 1, \pm 2, \ldots$.
C. $\varphi_{2}-\varphi_{1}=\left(2 n+\frac{1}{5}\right) \pi$ vói $n=0, \pm 1, \pm 2, \ldots$.
D. $\varphi_{2}-\varphi_{1}=\left(2 n+\frac{1}{3}\right) \pi$ với $n=0, \pm 1, \pm 2, \ldots$.

Assistant: {"answer":"B"}

--- Sample 2 (ID: MET_Phy_IE_2023_31) ---
Subject: Physics

User content:
Câu 31. Đặt điện áp xoay chiều có giá trị hiệu dụng $U$ và tần số không đổi vào hai đầu đoạn mạch gồm biến trở $R$, cuộn cảm thuần $L$ và tụ điện $C$ mắc nối tiếp. Khi $R=R_{1}$ thì điện áp hiệu dụng giữa hai đầu $L$ và hai đầu $C$ lần lượt là $U_{L}$ và $U_{C}$ với $U_{C}=2 U_{L}=U$. Khi $R=R_{2}=\frac{R_{1}}{\sqrt{3}}$ thì điện áp hiệu dụng 

In [8]:
# Kiểm tra phân phối đáp án
from collections import Counter

answers = []
for item in augmented_data:
    answer = json.loads(item["messages"][2]["content"])["answer"]
    answers.append(answer)

answer_counts = Counter(answers)
print("\nAnswer distribution:")
for answer, count in sorted(answer_counts.items()):
    percentage = count / len(answers) * 100
    print(f"{answer}: {count:5d} ({percentage:5.2f}%)")

print(f"\nTotal: {len(answers)}")


Answer distribution:
A:  1255 (26.59%)
B:  1147 (24.31%)
C:  1181 (25.03%)
D:  1136 (24.07%)

Total: 4719


## Tương tự cho các dataset khác

Có thể áp dụng cùng kỹ thuật cho:
- `val_sft.jsonl`
- `test_sft.jsonl`
- `train_sft_vmlu.jsonl`

In [9]:
# Augment validation set
val_input = Path("../data/sft_dataset_vnhsge/val_sft.jsonl")
val_output = Path("../data/sft_dataset_vnhsge/val_sft_augmented.jsonl")

if val_input.exists():
    print("Augmenting validation set...")
    augment_data(val_input, val_output, num_augmentations=1)
else:
    print(f"Validation file not found: {val_input}")

Augmenting validation set...
Loaded 4730 original samples


Augmenting data:   9%|▉         | 425/4730 [00:00<00:00, 47028.79it/s]


IndexError: list index out of range